# ATT&CK Techniques to Security Events

-----------------------------------

* **Author**: Jose Luis Rodriguez - [@Cyb3rPandaH](https://twitter.com/Cyb3rPandaH)
* **Organization**: [Open Threat Research (OTR)](https://github.com/OTRF)
* **References**: 
 - [OSSEM - ATT&CK Event Mapping](https://github.com/OTRF/OSSEM-DM/blob/main/docs/mitre_attack/techniques_to_events_mapping.yaml)
 - [Defining ATT&CK Data Sources, Part I: Enhancing the Current State](https://medium.com/mitre-attack/defining-attack-data-sources-part-i-4c39e581454f)
 - [Defining ATT&CK Data Sources, Part II: Operationalizing the Methodology](https://medium.com/mitre-attack/defining-attack-data-sources-part-ii-1fc98738ba5b)
 - [ATT&CK - Data Sources Definition](https://github.com/mitre-attack/attack-datasources/tree/main/contribution)

## Importing Python Libraries

In [1]:
# Importing library to manipulate data
import pandas as pd

# Importing library to manipulate yaml data
import yaml
import requests

# Importing library for visualizations
import plotly.graph_objects as go

## Importing (Sub)Techniques to Security Events Mapping Yaml File

Using the **[attackcti](https://attackcti.com/intro.html)** Python library, we were able to collect all the techniques within the Enterprise matrix. After that, we have mapped several security events to each data source/component/relationship. Here is the result:

In [2]:
yamlUrl = 'https://raw.githubusercontent.com/OTRF/OSSEM-DM/main/use-cases/mitre_attack/techniques_to_events_mapping.yaml'
yamlContent = requests.get(yamlUrl)
yamlMapping = yaml.safe_load(yamlContent.text)
mapping = pd.json_normalize(yamlMapping)
mapping.head()

,technique_id,x_mitre_is_subtechnique,technique,tactic,platform,data_source,data_component,name,source,relationship,target,event_id,event_name,event_platform,audit_category,audit_sub_category,log_channel,log_provider
0,T1553.006,True,Code Signing Policy Modification,[defense-evasion],"[Windows, macOS]",windows registry,windows registry key modification,Process modified Windows registry key,process,modified,windows registry key,13,RegistryEvent (Value Set).,Windows,RegistryEvent,NaN,Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon
1,T1553.006,True,Code Signing Policy Modification,[defense-evasion],"[Windows, macOS]",windows registry,windows registry key modification,Process modified Windows registry key,process,modified,windows registry key,14,RegistryEvent (Key and Value Rename).,Windows,RegistryEvent,NaN,Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon
2,T1553.006,True,Code Signing Policy Modification,[defense-evasion],"[Windows, macOS]",windows registry,windows registry key modification,Process modified Windows registry key,process,modified,windows registry key,4670,Permissions on an object were changed.,Windows,Object Access,Registry,Security,Microsoft-Windows-Security-Auditing
3,T1553.006,True,Code Signing Policy Modification,[defense-evasion],"[Windows, macOS]",windows registry,windows registry key modification,Process modified Windows registry key,process,modified,windows registry key,DeviceRegistryEvents,DeviceRegistryEvents,Windows,None,NaN,None,Windows Defender Advanced Threat Protection
4,T1553.006,True,Code Signing Policy Modification,[defense-evasion],"[Windows, macOS]",windows registry,windows registry key modification,Process modified Windows registry key value,process,modified,windows registry key value,13,RegistryEvent (Value Set).,Windows,RegistryEvent,NaN,Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon


### Which are the most relevant data sources for Sub-Technqiues within the Enterprise Matrix?

In [3]:
enterprise_sub = mapping[mapping['x_mitre_is_subtechnique'] == True][['technique_id','data_source','data_component']].drop_duplicates()
top15_data_source = enterprise_sub['data_source'].value_counts().nlargest(15)

fig = go.Figure()
fig.add_trace(go.Bar(y = top15_data_source.values.tolist(), x = top15_data_source.index.tolist(),orientation = 'v'))
fig.update_layout(title = {'text':"Count of sub-techniques per Data Source (Top 15)",'y': 0.85,'x': 0.5,'xanchor':'center'})
fig.show()

### Which are the most relevant data components for Sub-Technqiues within the Enterprise Matrix?

In [4]:
top15_data_component = enterprise_sub['data_component'].value_counts().nlargest(15)

fig = go.Figure()
fig.add_trace(go.Bar(y = top15_data_component.values.tolist(), x = top15_data_component.index.tolist(),orientation = 'v'))
fig.update_layout(title = {'text':"Count of sub-techniques per Data Component (Top 15)",'y': 0.85,'x': 0.5,'xanchor':'center'})
fig.show()

## Visualizing Relationships Among Data Elements: Network Graph

* Getting dataframe of relationships among data elements

In [5]:
relationships = mapping[['source','target','relationship']]\
      .drop_duplicates()\
      .dropna().reset_index(drop = True)
relationships.head()

,source,target,relationship
0,process,windows registry key,modified
1,process,windows registry key value,modified
2,user,windows registry key,modified
3,user,windows registry key value,modified
4,process,command,executed


* Importing libraries

In [6]:
import networkx as nx

from bokeh.io import output_notebook, show
from bokeh.models import (BoxSelectTool,LassoSelectTool,PanTool,BoxZoomTool,ResetTool,TapTool,
                          WheelZoomTool,HoverTool,Circle, MultiLine,Plot, Range1d,ColumnDataSource,
                          PointDrawTool,LabelSet,EdgesAndLinkedNodes, NodesAndLinkedEdges,HoverTool)
from bokeh.palettes import Spectral5
from bokeh.plotting import from_networkx

output_notebook()

Loading BokehJS ...

* Visualizing network graph

In [7]:
# Creating networkx graph object
G = nx.from_pandas_edgelist(relationships, source = 'source', target = 'target',edge_attr = 'relationship')

# Creating plot object
plot = Plot(plot_width=800, plot_height=500,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
plot.title.text = "Relationships among Data Elements"
plot.title.align = "center"
plot.title.text_font_size = "30px"

# Creating renderer objects
graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=60, fill_color=Spectral5[0])
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral5[3])
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral5[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color="gray", line_alpha=0.9, line_width=1)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral5[3], line_width=4)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral5[1], line_width=4)

graph_renderer.selection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = NodesAndLinkedEdges()
plot.renderers.append(graph_renderer)

# Adding nodes labels
x,y = zip(*graph_renderer.layout_provider.graph_layout.values())

names = []
for i in nx.nodes(G): names.append(i) 
    
nodes_table = ColumnDataSource({'x': x, 'y': y,'name': names})
labels = LabelSet(x='x',y='y',text='name',render_mode='canvas',y_offset=-10,text_align='center',
                  text_font_size='12px',source = nodes_table, background_fill_color=None,text_color = 'black')

plot.renderers.append(labels)

# Adding tools
plot.add_tools(HoverTool(tooltips=None),WheelZoomTool(),TapTool(),BoxSelectTool(),BoxZoomTool(),
               PanTool(),LassoSelectTool(),ResetTool())

# Showing graph
show(plot)

## Mapping (sub)techniques to data components & relationships & event logs

### Use Case: OS Credential Dumping: LSASS Memory (T1003.001)

* What are the recommended data sources?

In [8]:
mapping[mapping['technique_id']=='T1003.001'][['data_source', 'data_component']].drop_duplicates()

,data_source,data_component
3448,process,process creation
3455,process,process access
3457,command,command execution
3465,process,os api execution


* What security events have we identified so far??

In [9]:
mapping[mapping['technique_id']=='T1003.001']

,technique_id,x_mitre_is_subtechnique,technique,tactic,platform,data_source,data_component,name,source,relationship,target,event_id,event_name,event_platform,audit_category,audit_sub_category,log_channel,log_provider
3448,T1003.001,True,LSASS Memory,[credential-access],[Windows],process,process creation,Process created Process,process,created,process,4688,A new process has been created.,Windows,Detailed Tracking,Process Creation,Security,Microsoft-Windows-Security-Auditing
3449,T1003.001,True,LSASS Memory,[credential-access],[Windows],process,process creation,Process created Process,process,created,process,1,Process Creation.,Windows,ProcessCreate,NaN,Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon
3450,T1003.001,True,LSASS Memory,[credential-access],[Windows],process,process creation,Process created Process,process,created,process,DeviceProcessEvents,DeviceProcessEvents,Windows,None,NaN,None,Windows Defender Advanced Threat Protection
3451,T1003.001,True,LSASS Memory,[credential-access],[Windows],process,process creation,Process created Thread,process,created,thread,8,CreateRemoteThread.,Windows,CreateRemoteThread,NaN,Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon
3452,T1003.001,True,LSASS Memory,[credential-access],[Windows],process,process creation,User created Process,user,created,process,4688,A new process has been created.,Windows,Detailed Tracking,Process Creation,Security,Microsoft-Windows-Security-Auditing
3453,T1003.001,True,LSASS Memory,[credential-access],[Windows],process,process creation,User created Process,user,created,process,1,Process Creation.,Windows,ProcessCreate,NaN,Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon
3454,T1003.001,True,LSASS Memory,[credential-access],[Windows],process,process creation,User created Process,user,created,process,DeviceProcessEvents,DeviceProcessEvents,Windows,None,NaN,None,Windows Defender Advanced Threat Protection
3455,T1003.001,True,LSASS Memory,[credential-access],[Windows],process,process access,Process accessed Process,process,accessed,process,10,ProcessAccess.,Windows,ProcessAccess,NaN,Microsoft-Windows-Sysmon/Operational,Microsoft-Windows-Sysmon
3456,T1003.001,True,LSASS Memory,[credential-access],[Windows],process,process access,Process requested access to Process,process,requested access to,process,4656,A handle to an object was requested.,Windows,Object Access,Kernel Object,Security,Microsoft-Windows-Security-Auditing
3457,T1003.001,True,LSASS Memory,[credential-access],[Windows],command,command execution,Process executed Command,process,executed,command,4688,A new process has been created.,Windows,Detailed Tracking,Process Creation,Security,Microsoft-Windows-Security-Auditing


## This is really cool!!! We have covered most of the recommended data sources/components, however we have not identified security events for one data component: Process / OS Api Execution :(
    
## What security event logs would you recommend for this data source/component? Contribute to the community through OSSEM :D

## You can contribute and map more security events [here](https://github.com/OTRF/OSSEM-DM/tree/main/relationships)